In [57]:
import os
import numpy as np
import pandas as pd
import pickle
import itertools
from matplotlib import pyplot as plt
from datetime import date, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.stats import norm
from sklearn.inspection import permutation_importance
from importlib import reload
import matplotlib.pyplot as plt
from datetime import date, timedelta
from matplotlib import rc
from sklearn.model_selection import train_test_split
import tqdm

In [58]:
data = pd.read_csv('Data/statsmodels rel.csv', sep=';', index_col='Date', parse_dates=['Date']).drop('Unnamed: 0', axis=1)
display(data.tail())

,No_cases,Workplace,Events,Daycare,Higher Education,Secondary Education,Primary Education,Restaurants/Cafes,Transport,Entertainment,...,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken,Rolling_R
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-02,0.457531,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.926207,0.907408
2021-05-03,0.454589,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.943694,0.893238
2021-05-04,0.424368,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.960529,0.880369
2021-05-05,0.472085,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.980234,0.869616
2021-05-06,0.426674,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,1.000000,0.860512


In [59]:
def shift_outcome(X,y,s,shift_date="y"):

    # If shift_date="y", then the date of y is shifted s days into the past. If shift_date="X", then the date of X is shifted s days into the future.

        if shift_date=="y":
            
            y_shifted = y.reset_index().copy()
            y_shifted["Date"] = y_shifted[["Date"]].applymap(lambda x: x.date()-timedelta(s))
            y_shifted = y_shifted.set_index(["Date"])
            y_shifted.drop(y_shifted.loc[np.isnan(y_shifted.values)].index,inplace=True)
            data2 = X.join(y_shifted,how="inner")
            X = data2[X.columns]
            y_shifted = data2[y.name]
            
            return X,y_shifted
        
        if shift_date=="X":
        
            X_shifted = X.reset_index().copy()
            X_shifted["Date"] = X_shifted[["Date"]].applymap(lambda x: x.date()+timedelta(s))
            X_shifted = X_shifted.set_index(["Date"])
            y.drop(y.loc[np.isnan(y.values)].index,inplace=True)
            data = X_shifted.join(y,how="inner")
            X_shifted = data[X.columns]
            y = data[y.name]
            
            return X_shifted,y

In [60]:
%%time
X = data.drop(['No_cases', 'Rolling_R'], axis=1)
Y = data['Rolling_R']

#X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
results = pd.DataFrame()

timeshift = range(0, 11)
min_samples_leaf = [1]
max_tree_depth = [d for d in range(1,16)]
max_features = [n/100 for n in range(1,101, 10)]
max_features.append(1)
n_estimators = 100
maxsamples = 3/4
kfold_splits = 5

for s in timeshift:
    X_used, Y_used = shift_outcome(X,Y,s,shift_date="y")
#    X_used, Y_used = cut_dates(X_used,Y_used,enddate = date(2021,4,30))
    for m in min_samples_leaf:
        for d in max_tree_depth:
            print(f'timeshift: {s}. tree depth: {d}')
            for n in max_features:
                kf = KFold(n_splits=kfold_splits, shuffle=True)

                for train_ind, test_ind in kf.split(X_used):
                    X_train = X_used.iloc[train_ind]
                    Y_train = Y_used.iloc[train_ind]
                    X_test = X_used.iloc[test_ind]
                    Y_test = Y_used.iloc[test_ind]

                    forest = RandomForestRegressor(
                                min_samples_leaf=m,
                                max_depth=d,
                                max_features=n,
                                n_estimators=n_estimators,
                                max_samples=maxsamples,
                                bootstrap=True).fit(X_train.values,Y_train.values.ravel())

                    # compute several metrics for the goodness of the fit
                    R2_train  = forest.score(X_train.values,Y_train.values.ravel())
                    MSE_train = mean_squared_error(forest.predict(X_train.values),Y_train.values.ravel())
                    RSS_train = np.sum((forest.predict(X_train.values)-Y_train.values.ravel())**2)

                    R2_test  = forest.score(X_test.values,Y_test.values.ravel())
                    MSE_test = mean_squared_error(forest.predict(X_test.values),Y_test.values.ravel())
                    RSS_test = np.sum((forest.predict(X_test.values)-Y_test.values.ravel())**2)

                    results = results.append(pd.DataFrame(
                                {'timeshift':           [s],
                                'min_samples_leaf':     [m],
                                'max_tree_depth':       [d],
                                'max_features':         [n],
                                'R2_test':              [R2_test],
                                'MSE_test':             [MSE_test],
                                'RSS_test':             [RSS_test],
                                'n_datapoints_test':    [len(Y_test)],
                                'R2_train':             [R2_train],
                                'MSE_train':            [MSE_train],
                                'RSS_train':            [RSS_train],
                                'n_datapoints_train':   [len(Y_train)]
                            }),ignore_index=False)

    results

timeshift: 0. tree depth: 1
timeshift: 0. tree depth: 2
timeshift: 0. tree depth: 3
timeshift: 0. tree depth: 4
timeshift: 0. tree depth: 5
timeshift: 0. tree depth: 6
timeshift: 0. tree depth: 7
timeshift: 0. tree depth: 8
timeshift: 0. tree depth: 9
timeshift: 0. tree depth: 10
timeshift: 0. tree depth: 11
timeshift: 0. tree depth: 12
timeshift: 0. tree depth: 13
timeshift: 0. tree depth: 14
timeshift: 0. tree depth: 15
timeshift: 1. tree depth: 1
timeshift: 1. tree depth: 2
timeshift: 1. tree depth: 3
timeshift: 1. tree depth: 4
timeshift: 1. tree depth: 5
timeshift: 1. tree depth: 6
timeshift: 1. tree depth: 7
timeshift: 1. tree depth: 8
timeshift: 1. tree depth: 9
timeshift: 1. tree depth: 10
timeshift: 1. tree depth: 11
timeshift: 1. tree depth: 12
timeshift: 1. tree depth: 13
timeshift: 1. tree depth: 14
timeshift: 1. tree depth: 15
timeshift: 2. tree depth: 1
timeshift: 2. tree depth: 2
timeshift: 2. tree depth: 3
timeshift: 2. tree depth: 4
timeshift: 2. tree depth: 5
timeshif

In [61]:
performance = results[["timeshift","min_samples_leaf","max_tree_depth","max_features","n_datapoints_test","R2_test","RSS_test","R2_train","RSS_train","n_datapoints_train","MSE_test","MSE_train"]].groupby(['timeshift','min_samples_leaf','max_tree_depth',"max_features"],as_index=False).agg(['mean','sum'])
performance["RSS_test/n_datapoints_test"] = performance[("RSS_test",'sum')]/performance[("n_datapoints_test",'sum')]
performance["RSS_train/n_datapoints_train"] = performance[("RSS_train",'sum')]/performance[("n_datapoints_train",'sum')]
performance


n_datapoints_test       \
                                                                    mean  sum   
timeshift min_samples_leaf max_tree_depth max_features                          
0         1                1              0.01                      87.4  437   
                                          0.11                      87.4  437   
                                          0.21                      87.4  437   
                                          0.31                      87.4  437   
                                          0.41                      87.4  437   
...                                                                  ...  ...   
10        1                15             0.61                      85.4  427   
                                          0.71                      85.4  427   
                                          0.81                      85.4  427   
                                          0.91                      85.4  427   
                                          1.00                      85.4  427   

                                                         R2_test            \
                                                            mean       sum   
timeshift min_samples_leaf max_tree_depth max_features                       
0         1                1              0.01          0.399347  1.996737   
                                          0.11          0.537164  2.685818   
                                          0.21          0.608145  3.040723   
                                          0.31          0.583591  2.917957   
                                          0.41          0.480719  2.403593   
...                                                          ...       ...   
10        1                15             0.61          0.660323  3.301616   
                                          0.71          0.667895  3.339477   
                                          0.81          0.654944  3.274719   
                                          0.91          0.673980  3.369901   
                                          1.00          0.623190  3.115950   

                                                        RSS_test             \
                                                            mean        sum   
timeshift min_samples_leaf max_tree_depth max_features                        
0         1                1              0.01          3.889816  19.449080   
                                          0.11          3.000290  15.001452   
                                          0.21          2.468365  12.341823   
                                          0.31          2.457515  12.287576   
                                          0.41          2.788965  13.944823   
...                                                          ...        ...   
10        1                15             0.61          0.943919   4.719594   
                                          0.71          0.916381   4.581906   
                                          0.81          0.963973   4.819867   
                                          0.91          0.912598   4.562988   
                                          1.00          1.055520   5.277602   

                                                        R2_train            \
                                                            mean       sum   
timeshift min_samples_leaf max_tree_depth max_features                       
0         1                1              0.01          0.402691  2.013453   
                                          0.11          0.541803  2.709013   
                                          0.21          0.617447  3.087233   
                                          0.31          0.619532  3.097662   
                                          0.41          0.612680  3.063402   
...                                                          ...       ...   
10        1       

In [63]:
# manually select best
performance.sort_values(by=('R2_test', 'mean'), ascending=False)

n_datapoints_test       \
                                                                    mean  sum   
timeshift min_samples_leaf max_tree_depth max_features                          
0         1                9              0.21                      87.4  437   
                           13             0.71                      87.4  437   
                                          0.01                      87.4  437   
                           10             0.41                      87.4  437   
1         1                13             0.41                      87.2  436   
...                                                                  ...  ...   
8         1                1              0.01                      85.8  429   
10        1                1              0.01                      85.4  427   
9         1                1              1.00                      85.6  428   
10        1                1              1.00                      85.4  427   
9         1                1              0.01                      85.6  428   

                                                         R2_test            \
                                                            mean       sum   
timeshift min_samples_leaf max_tree_depth max_features                       
0         1                9              0.21          0.810731  4.053653   
                           13             0.71          0.810670  4.053351   
                                          0.01          0.808902  4.044510   
                           10             0.41          0.808790  4.043950   
1         1                13             0.41          0.808743  4.043713   
...                                                          ...       ...   
8         1                1              0.01          0.295542  1.477710   
10        1                1              0.01          0.291369  1.456845   
9         1                1              1.00          0.286543  1.432715   
10        1                1              1.00          0.269565  1.347823   
9         1                1              0.01          0.263691  1.318455   

                                                        RSS_test             \
                                                            mean        sum   
timeshift min_samples_leaf max_tree_depth max_features                        
0         1                9              0.21          1.213379   6.066897   
                           13             0.71          1.238471   6.192357   
                                          0.01          1.246137   6.230686   
                           10             0.41          1.228827   6.144136   
1         1                13             0.41          1.062099   5.310494   
...                                                          ...        ...   
8         1                1              0.01          2.176602  10.883011   
10        1                1              0.01          2.001260  10.006298   
9         1                1              1.00          2.134451  10.672255   
10        1                1              1.00          2.074244  10.371221   
9         1                1              0.01          2.148292  10.741460   

                                                        R2_train            \
                                                            mean       sum   
timeshift min_samples_leaf max_tree_depth max_features                       
0         1                9              0.21          0.818806  4.094028   
                           13             0.71          0.820093  4.100463   
                                          0.01          0.820184  4.100921   
                           10             0.41          0.820509  4.102544   
1         1                13             0.41          0.822248  4.111239   
...                                                          ...       ...   
8         1       

In [79]:
n_repeats = 5
n_splits = 10

timeshift = [0]

permutation_importance = pd.DataFrame()

# optimal parameters per timeshift
for s in timeshift:
    temp = performance.loc[s][('R2_test','mean')].idxmax()
    min_samples_leaf = temp[0] #m
    max_tree_depth = 13 #temp[1] #d
    max_features = 0.71 #temp[2] #n
    
    X = data.drop(['No_cases', 'Rolling_R'], axis=1)
    Y = data['Rolling_R']
    X_used,Y_used = shift_outcome(X,Y,s,shift_date="y")
    
    forest = RandomForestRegressor(max_depth = max_tree_depth, max_features = max_features,
                                   min_samples_leaf = min_samples_leaf, max_samples = 3/4)
    
    permutation_scores = pd.DataFrame()
    print(X.columns)
    for feature in tqdm.tqdm(X.columns):
        
        Deltas = []

        kf = RepeatedKFold(n_splits=n_splits,n_repeats=n_repeats, random_state=0)

        for train_index, test_index in kf.split(X_used):

            X1 = X_used.copy()
            v = X1[feature].values
            np.random.shuffle(v)
            X1[feature] = v
#             X1[feature] = np.random.uniform(low=0, high=1, size=len(v))

            X_train = X_used.iloc[train_index,:]
            y_train = Y_used.iloc[train_index]
            
            X_test = X_used.iloc[test_index,:]
            y_test = Y_used.iloc[test_index]
            
            # change one feaure with disturbance
            X1_test = X_test.copy()
            X1_test[feature] = np.random.uniform(low=0, high=1, size=len(X1_test))
            
            
            trained_forest = forest.fit(X_train, y_train.values.ravel())

            score = trained_forest.score(X1_test.values, y_test.values.ravel())
            baseline = trained_forest.score(X_test.values,y_test.values.ravel())

            # high deltas mean bad model after, mean high importance
            Deltas = Deltas + [baseline-score]
            

#         mean = np.array(Deltas).mean()
#         std = np.array(Deltas).std()/np.sqrt(n_repeats*n_splits)
        
        relatief = pd.DataFrame(Deltas).quantile([0.5, .025, 0.975]).T
        relatief.columns = ['median', 'low', 'high']
        relatief['Measure'] = feature
        permutation_scores = permutation_scores.append(relatief)
#         permutation_scores = permutation_scores.append(
#             pd.DataFrame({
#                 'Measure':[feature],
#                 'Deltas':[np.array(Deltas)],
#                 'mean_Delta': [mean],
#                 'std_Delta':[std]
#             }))

    permutation_scores = permutation_scores.sort_values('median',ascending=False)        
    permutation_scores["Rank"] = np.arange(1,permutation_scores.shape[0]+1)
    permutation_scores = permutation_scores.set_index("Rank",drop=False)
    permutation_scores['timeshift'] = s
    permutation_importance = permutation_importance.append(permutation_scores, ignore_index=False)

  0%|          | 0/21 [00:00<?, ?it/s]

Index(['Workplace', 'Events', 'Daycare', 'Higher Education',
       'Secondary Education', 'Primary Education', 'Restaurants/Cafes',
       'Transport', 'Entertainment', 'Sports', 'Indoor', 'Masks',
       'MassGather50', 'MassGatherAll', 'Shops', 'Outdoor', 'Religion',
       'Gatherings', 'Stayhome', 'Telework', 'Prikken'],
      dtype='object')


100%|██████████| 21/21 [02:56<00:00,  8.39s/it]


In [72]:
performance.loc[0][('R2_test','mean')].idxmax()

(1, 9, 0.21)

In [80]:
display(permutation_importance)

,median,low,high,Measure,Rank,timeshift
Rank,,,,,,
1,0.320363,0.069381,1.572198,Restaurants/Cafes,1,0
2,0.293688,0.053960,1.358480,Entertainment,2,0
3,0.278115,0.066485,0.903793,MassGatherAll,3,0
4,0.259556,0.065505,1.278198,Telework,4,0
5,0.110081,0.040048,0.425484,Sports,5,0
6,0.096489,0.010634,0.256089,Outdoor,6,0
7,0.037495,0.007749,0.131850,Prikken,7,0
8,0.032621,0.008838,0.113599,Daycare,8,0
9,0.020484,-0.013156,0.104098,Masks,9,0


In [81]:
permutation_importance.to_csv('RF perm imp rel.csv', sep=';', index=False)